In [ ]:
# Add your client ID, callback URL and Evo scopes here.
# NOTE: Add the scope "offline_access" to the list if you require a refresh token.

client_id = "your-client-id"
callback_url = ""
scope = ["evo.workspaces", "evo.discovery", "evo.object", "evo.blocksync"]

In [ ]:
import datetime
import webbrowser
from http.server import BaseHTTPRequestHandler, HTTPServer
from urllib.parse import urlparse

import jwt
from authlib.common.security import generate_token
from authlib.integrations.requests_client import OAuth2Session


def get_access_token(
    client_id: str,
    redirect_uri: str,
    authorisation_url: str,
    access_token_url: str,
    scope: str,
) -> str:
    """Get an access token."""

    port = urlparse(redirect_uri).port

    class OAuthHttpServer(HTTPServer):
        def __init__(self, *args, **kwargs):
            super().__init__(*args, **kwargs)
            self.authorization_code = ""

    class OAuthHttpHandler(BaseHTTPRequestHandler):
        def do_GET(self):
            self.send_response(200)
            self.send_header("Content-Type", "text/html")
            self.end_headers()
            self.wfile.write("Redirecting to the Bentley login page\n".encode("UTF-8"))

            client.fetch_token(
                url=access_token_url,
                authorization_response=self.path,
                state=state,
                code_verifier=code_verifier,
                grant_type="authorization_code",
            )

            self.wfile.write(
                """
                <html>
                  <body>
                    <h2>Authorization request to Bentley has been completed.</h1>
                    <h3>You may now close this tab or window now.</h3>
                  </body>
                </html>
                """.encode("UTF-8")
            )
            self.wfile.write(
                '<script> setTimeout("window.close()", 2500);</script>'.encode("UTF-8")
            )  # Timeout only works if already logged in

    with OAuthHttpServer(("", port), OAuthHttpHandler) as httpd:
        client = OAuth2Session(
            client_id=client_id,
            scope=scope,
            redirect_uri=redirect_uri,
            code_challenge_method="S256",
        )

        code_verifier = generate_token(48)
        auth_uri, state = client.create_authorization_url(authorisation_url, code_verifier=code_verifier)
        webbrowser.open_new(auth_uri)
        httpd.handle_request()

    return client.token["access_token"]


token = get_access_token(
    client_id=client_id,
    redirect_uri=callback_url,
    authorisation_url="https://ims.bentley.com/connect/authorize",
    access_token_url="https://ims.bentley.com/connect/token",
    scope=scope,
)

print("Access token:")
print(token)

decoded = jwt.decode(token, options={"verify_signature": False}, algorithms=["RS256"])
exp_timestamp = decoded["exp"]
exp_datetime = datetime.datetime.fromtimestamp(exp_timestamp, datetime.timezone.utc)
print(f"Token expires at:\n{exp_datetime} UTC")